# 0.0. Imports

In [1]:
import re
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sqlalchemy import create_engine

from umap.umap_ import UMAP
from scipy.cluster import hierarchy as hc

from sklearn import cluster
from sklearn import metrics
from sklearn import preprocessing as pp
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import ensemble as en
from sklearn.mixture import GaussianMixture as gm

## 0.1. Helper Functions

In [2]:
def descriptive_statistics(num_attr):
    # Central Tendency: mean, median
    c1 = pd.DataFrame(num_attr.apply(np.mean))
    c2 = pd.DataFrame(num_attr.apply(np.median))

    # Dispension: min, max, range, std, skew, kurtosis
    d1 = pd.DataFrame(num_attr.apply(min))
    d2 = pd.DataFrame(num_attr.apply(max))
    d3 = pd.DataFrame(num_attr.apply(lambda x: x.max() - x.min()))
    d4 = pd.DataFrame(num_attr.apply(lambda x: x.std()))
    
    # Measures of Shape
    s1 = pd.DataFrame(num_attr.apply(lambda x: x.skew()))
    s2 = pd.DataFrame(num_attr.apply(lambda x: x.kurtosis()))

    # concat
    m = pd.concat([d1,d2,d3,c1,c2,d4,s1,s2], axis=1).reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    return m

## 0.2. Load Data

In [3]:
import s3fs

In [4]:
# path = '/home/cid/repos/clustering-high-value-customers-identification/'
path_s3 = 'https://insiders-dataset-cid.s3.amazonaws.com/'

df_raw = pd.read_csv(path_s3 + 'Ecommerce.csv', encoding='latin1')

# drop extra column
df_raw = df_raw.drop('Unnamed: 8', axis=1)


# 1.0. Data Description

In [5]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [6]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df1.columns = cols_new

## 1.2. Data Dimnesions

In [7]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

Number of Rows: 541909
Number of Columns: 8


## 1.3. Data Types

In [8]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [9]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5. Replace NA

In [10]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [11]:
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge 
df1 = pd.merge(df1, df_backup, how='left', on='invoice_no' )

# coalesce 
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

df1 = df1.drop(['customer_id_x', 'customer_id_y'], axis=1)

In [12]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6. Change dtypes

In [13]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

In [14]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])
df1['customer_id'] = df1['customer_id'].astype(int)

## 1.7. Descriptive Statistics

In [15]:
num_att = df1.select_dtypes(include=['int64', 'float64'])
cat_att = df1.select_dtypes(include=['object'])

## 1.7.1. Numerical Attributes

In [16]:
descriptive_statistics(num_att)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.081158,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759853,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.411352,0.487449,-0.804287


## 1.7.2. Categorical Attributes

In [17]:
cat_att.describe(include=['O'])

,invoice_no,stock_code,description,country
count,541909,541909,540455,541909
unique,25900,4070,4223,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,1114,2313,2369,495478


# 2.0. Data Filtering

In [18]:
df2 = df1.copy()

## 2.1. Filter Columns

In [19]:
cols_drop = ['description']
df2 = df2.drop(cols_drop, axis=1)

## 2.2. Filter Rows

In [20]:
# Numerical Attributes
df2 = df2.loc[df2['unit_price'] >= 0.4, :]

# Categorical Attributes
df2 = df2.loc[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] ), :]

# map
df2 = df2.loc[~df2['country'].isin(['European Community', 'Unspecified' ]), :]

# bad user
df2 = df2[~df2['customer_id'].isin( [16446] )]

# quantity
df2_returns   = df2.loc[df2['quantity'] < 0, :]
df2_purchases = df2.loc[df2['quantity'] >= 0, :]

# 3.0. Feature Engineering

In [21]:
df3 = df2.copy()

## 3.1. Feature Creation

In [22]:
drop_cols = ['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country']
df_ref = df3.drop(drop_cols, axis=1).drop_duplicates(ignore_index=True)

In [23]:
df2_purchases.loc[:, ['gross_revenue']] = (df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price'])

/home/cid/.pyenv/versions/3.8.0/envs/clustering-high-value-customers-identification/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### 3.1.1. Gross Revenue

In [24]:
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index() # .rename(columns={'gross_revenue': 'monetary'})
df_ref = pd.merge(df_ref, df_monetary, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    92
dtype: int64

### 3.1.2. Recency

In [25]:
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df_recency['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_ref = pd.merge(df_ref, df_recency[['customer_id', 'recency_days']], how='left', on='customer_id')

df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
dtype: int64

### 3.1.3. Quatily of purchased

In [26]:
df_freq = df2_purchases[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().\
rename(columns={'invoice_no': 'qtde_invoices'})

df_ref = pd.merge(df_ref, df_freq, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
qtde_invoices    92
dtype: int64

### 3.1.4. Quantity total of items purchased

In [27]:
df_freq = (df2_purchases.loc[:, ['customer_id', 'quantity']].groupby('customer_id')
                                                            .sum()
                                                            .reset_index()
                                                            .rename(columns={'quantity': 'qtde_items'}))
df_ref = pd.merge(df_ref, df_freq, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
qtde_invoices    92
qtde_items       92
dtype: int64

### 3.1.5. Quantity of products purchased

In [28]:
df_freq = ( df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id')
                                                               .count()
                                                               .reset_index()
                                                               .rename(columns={'stock_code': 'qtde_products'}))
df_ref = pd.merge(df_ref, df_freq, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
qtde_invoices    92
qtde_items       92
qtde_products    92
dtype: int64

### 3.1.6. Average Ticket

In [29]:
df_avg_ticket =  (df2_purchases.loc[:, ['customer_id','gross_revenue']].groupby('customer_id')
                                                                       .mean()
                                                                       .reset_index()
                                                                       .rename(columns={'gross_revenue': 'avg_ticket'}))

df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')

df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
qtde_invoices    92
qtde_items       92
qtde_products    92
avg_ticket       92
dtype: int64

### 3.1.7. Average Recency Days

In [30]:
# df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False, False])
# df_aux['next_customer_id'] = df_aux['customer_id'].shift()
# df_aux['previus_date'] = df_aux['invoice_date'].shift()

# df_aux['avg_recency_days'] = df_aux.apply( lambda x: (x['invoice_date'] - x['previus_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)
# df_aux['avg_recency_days'] = df_aux['avg_recency_days'] * -1
# df_aux = df_aux.drop(columns=['invoice_date', 'next_customer_id', 'previus_date'], axis=1).dropna()

# df_avg_recency_days = df_aux.groupby( 'customer_id' ).mean().reset_index()

# df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
# df_ref.isna().sum()

### 3.1.8. Frequency Purchase

In [31]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                 .groupby('customer_id')
                                                 .agg( max_ = ('invoice_date', 'max'),
                                                       min_ = ('invoice_date', 'min'),
                                                       days = ('invoice_date', lambda x: ((x.max() - x.min()).days) + 1 ),
                                                       buy_ = ( 'invoice_no', 'count' ))).reset_index()

df_aux['frequency'] = df_aux[['buy_', 'days']].apply( lambda x: x['buy_'] / x['days'] if x['days'] != 0 else 0, axis=1 )

df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
qtde_invoices    92
qtde_items       92
qtde_products    92
avg_ticket       92
frequency        92
dtype: int64

### 3.1.9. Number Or Returns

In [32]:
df_returns = df2_returns[['quantity', 'customer_id']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, on='customer_id', how='left')
df_ref['qtde_returns'].fillna(0, inplace=True)
df_ref.isna().sum()

customer_id       0
gross_revenue    92
recency_days     92
qtde_invoices    92
qtde_items       92
qtde_products    92
avg_ticket       92
frequency        92
qtde_returns      0
dtype: int64

### 3.1.10. Basket Size

In [33]:
df_aux = (df2_purchases[['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                   .agg( n_purchase=('invoice_no', 'nunique'),
                                                                         n_products=('quantity', 'sum'))).reset_index()
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']
df_ref = pd.merge(df_ref, df_aux[['avg_basket_size', 'customer_id']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id         0
gross_revenue      92
recency_days       92
qtde_invoices      92
qtde_items         92
qtde_products      92
avg_ticket         92
frequency          92
qtde_returns        0
avg_basket_size    92
dtype: int64

### 3.1.11. Unique Basket Size

In [34]:
df_aux = (df2_purchases.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                         .agg(n_purchase=('invoice_no', 'nunique'),
                                                                              n_products=('stock_code', 'nunique'))).reset_index()
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_ref = pd.merge(df_ref, df_aux[['avg_unique_basket_size', 'customer_id']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id                0
gross_revenue             92
recency_days              92
qtde_invoices             92
qtde_items                92
qtde_products             92
avg_ticket                92
frequency                 92
qtde_returns               0
avg_basket_size           92
avg_unique_basket_size    92
dtype: int64

# 4.0. EDA  

In [35]:
df_ref = df_ref.dropna()

df4 = df_ref.copy()

## 4.3. Space Study

In [36]:
# select dataser
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_products', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].copy()

mms = pp.MinMaxScaler()

df43['gross_revenue'] = mms.fit_transform( df43[['gross_revenue']].values )
df43['recency_days'] = mms.fit_transform( df43[['recency_days']].values )
df43['qtde_products'] = mms.fit_transform( df43[['qtde_products']].values )
df43['frequency'] = mms.fit_transform( df43[['frequency']].values )
df43['qtde_returns'] = mms.fit_transform( df43[['qtde_returns']].values )

### 4.3.4. Tree-Based Embbedding

In [37]:
X = df43.drop(columns=['customer_id', 'gross_revenue'])
y = df43['gross_revenue']

# model training
rf_model = en.RandomForestRegressor( n_estimators=100, random_state=42 )

# model training definition
rf_model.fit( X, y)

#  leaf
df_leaf = pd.DataFrame( rf_model.apply( X ) )

# create dataframe tree
df_tree = pd.DataFrame()

# reduzer dimensionality
reducer = UMAP(random_state=42)
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

/home/cid/.pyenv/versions/3.8.0/envs/clustering-high-value-customers-identification/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


# 5.0. Data Preparation

In [38]:
df5 = df_tree.copy()
# df5.to_csv( '../src/data/tree_based_embedding.csv', index=False )

# 7.0. Hyperpameter Fine Tuning

In [39]:
X = df5.copy()

X.head()

,embedding_x,embedding_y
0,17.881968,14.895823
1,17.940453,12.172152
2,18.596876,12.517838
3,-6.055115,-13.710513
4,5.152057,18.690804


# 8.0. Model Training

In [40]:
# model definition
k = 8
gmm_model = gm(n_components=k, n_init=100, random_state=42)

# model training
gmm_model.fit(X)

# model predict
labels = gmm_model.predict(X)

## 8.2. Cluster Validation 

In [41]:
print("SS value: {}".format(metrics.silhouette_score( X, labels, metric='euclidean' )))

SS value: 0.37607449293136597


# 9.0. Cluster Analysis

In [42]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels

# change dtypes
df92['recency_days'] = df92['recency_days'].astype(int)
df92['qtde_products'] = df92['qtde_products'].astype(int)
df92['qtde_returns'] = df92['qtde_returns'].astype(int)

## 9.2. Cluster Profile

In [43]:
# cluster - qt_users - per_user
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index().rename(columns={'customer_id': 'qt_users'})
df_cluster['per_user'] = 100 * (df_cluster['qt_users'] / df_cluster['qt_users'].sum())


# gross_revenue
monetary = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, monetary, how='left', on='cluster')

# recency_days
recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, recency_days, how='left', on='cluster')

# qtde_products
qtde_products = df92[['qtde_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, qtde_products, how='left', on='cluster')

# frequency
frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, frequency, how='left', on='cluster')

# qtde_returns
qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, qtde_returns, how='left', on='cluster')


df_cluster.sort_values('gross_revenue', ascending=False).style.highlight_max( color='lightgreen', axis=0 )

,cluster,qt_users,per_user,gross_revenue,recency_days,qtde_products,frequency,qtde_returns
0,0,918,16.159127,5890.422756,55.006536,292.521786,0.202352,162.967320
7,7,729,12.832248,2099.885254,101.890261,158.170096,0.469244,0.185185
3,3,363,6.389720,1167.181322,61.900826,57.005510,0.027708,9.991736
6,6,219,3.854955,959.385160,81.625571,38.721461,0.232266,11.292237
2,2,980,17.250484,928.779551,146.681633,51.101020,0.751629,5.207143
5,5,1046,18.412251,747.913948,167.919694,24.534417,0.963410,3.863289
4,4,755,13.289914,506.806265,90.627815,16.890066,0.411353,1.815894
1,1,671,11.811301,404.091148,167.336811,12.126677,0.696441,5.403875


In [44]:
# 1 Cluster Insiders
# 5 Cluster More Products
# 4 Cluster Spend Money 
# 2 Cluster Even More Products
# 6 Cluster Less Days
# 0 Cluster Less 1k
# 7 Cluster Stop Returners
# 3 Cluster More Buy

**Cluster 01:  ( Candidato à Insider )**
- Número de customers: 468 (16% do customers )
- Faturamento médio: 8836
- Recência média: 21 dias
- Média de Produtos comprados: 424 produtos
- Frequência de Produtos comprados: 0.09 produtos/dia
- Receita em média: $8836.13,00 dólares

# 10.0. EDA

In [45]:
df10 = df92.copy()

# 11.0. Deploy to Product

In [46]:
df11 = df10.copy()

## 11.1. Insert into SQLITE

In [47]:
# create table
query_create_table_insiders = """
    CREATE TABLE insiders (
        customer_id        INTEGER,
        gross_revenue      REAL,
        recency_days       INTEGER,
        qtde_products      INTEGER,
        frequency          REAL,
        qtde_returns       INTEGER,
        cluster            INTEGER
    )
"""
# conn = sqlite3.connect( 'insiders_db.sqlite' )
# conn.execute( query_create_table_insiders )
# conn.commit()
# conn.close()

In [48]:
# Drop Table
query_drop_table = """
    DROP TABLE  insiders
"""

In [49]:
# insert data
conn = create_engine( 'sqlite:///insiders_db.sqlite' )
df92.to_sql( 'insiders', con=conn,  if_exists='append', index=False )

In [50]:
# consulting database
query = """
    SELECT * FROM insiders
"""
conn = create_engine( 'sqlite:///insiders_db.sqlite' )

df  = pd.read_sql_query( query, conn )
df.head()

,customer_id,gross_revenue,recency_days,qtde_products,frequency,qtde_returns,cluster
0,17850,5391.21,372,297,17.000000,40,0
1,13047,3232.59,56,171,0.028302,35,0
2,12583,6495.30,2,221,0.040323,50,0
3,13748,938.89,95,27,0.017921,0,5
4,15100,876.00,333,3,0.073171,22,4


 ## Metabase 

1. download https://www.metabase.com/docs/latest/operations-guide/running-the-metabase-jar-file.html
* java -jar metabase.jar
* http://localhost:3000


 ## Banco de Dados na AWS
 
1. Aula 047 - Ciclo 11- Deploy do modelo em produção - Parte III
2. começa em 1h00

 ## pip install papermill
1. pip install papermill
2. papermill src/models/c12-deploy.ipynb reports/c12-deploy.ipynbc12-deploy_2021_10_13.ipynb

 ## bash
3. echo $(date +'%Y-%m-%d %H:%M:%S')
4. bash model_run.sh